# examples for different achitectures of neuronal networks

In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn import Linear

## Multilayer Perceptron Implementation
## predicts configuration and assumes that e.g. Carbon atom chains are in 3D space and of length 20 atoms ->(input_dim=60)

In [2]:
class MLP(nn.Module):
    
    def __init__(self):
        super(MLP, self).__init__()
        
        self.in_size = 60       #len(inputs[0,:])
        self.hidden1_size = 64
        self.hidden2_size = 128
        self.hidden3_size = 64
        self.hidden4_size = 32
        self.out_size = 1
        
        self.hidden1 = nn.Linear(self.in_size,self.hidden1_size)
        self.hidden2 = nn.Linear(self.hidden1_size,self.hidden2_size)
        self.hidden3 = nn.Linear(self.hidden2_size,self.hidden3_size)
        self.hidden4 = nn.Linear(self.hidden3_size,self.hidden4_size)
        
        self.out = nn.Linear(self.hidden4_size,self.out_size)
        
    def forward(self,inputs):
        
        inputs = nn.functional.elu(self.hidden1(inputs))
        inputs = nn.functional.elu(self.hidden2(inputs))
        inputs = nn.functional.elu(self.hidden3(inputs))
        inputs = nn.functional.elu(self.hidden4(inputs))
        out = self.out(inputs)
        out = out.view(-1)
        
        return out

model = MLP()
print(model)

MLP(
  (hidden1): Linear(in_features=60, out_features=64, bias=True)
  (hidden2): Linear(in_features=64, out_features=128, bias=True)
  (hidden3): Linear(in_features=128, out_features=64, bias=True)
  (hidden4): Linear(in_features=64, out_features=32, bias=True)
  (out): Linear(in_features=32, out_features=1, bias=True)
)


## Batchwise data augmentation 
### as it wasn't further specified, this implementation requires to concatenate original data and new augemented data afterwards (if wanted) to increase the total data amount for later training

In [3]:
def prepare_batch(data, augment_rotation: bool, augment_permutation: bool):
    # data is a numpy array with shape (batch_size, 20, 3)
    out = data.copy()
    batch_size = len(data[:,0,0])
    original_list = np.arange(20)
    indice_list = np.arange(20)
    if augment_rotation:
        
        for i in range(batch_size):
            M = np.random.normal(size=(3, 3))
            rotation_matrix = np.linalg.qr(M)[0] #rot. matrix creation
            for j in range(20):
                out[i][j] = rotation_matrix.dot(out[i,j,:]) #overwriting
                
        
    if augment_permutation:
        
        for n in range(batch_size):
            np.random.shuffle(indice_list) # new indice ordering of indice_list
            for m in range(20):
                original_indice = original_list[m]
                out[n][original_indice] = out[n,indice_list[m],:] #overwriting
    # return augmented data with shape (batch_size, 60)
    out = torch.from_numpy(out.astype(np.float32))
    out = out.view(batch_size,60) #providing shape of input that is later required
    return out

## Some testing on random data

In [4]:
#defining the data for  test:
data_np = np.random.rand(10,20,3)
print(data_np.shape)
# performing data rotation and permutation
output_np = prepare_batch(data_np, augment_rotation = True, augment_permutation = True)
print(output_np.shape)
print(type(output_np))
# Applys data to model and receives configurations according
# to number of data sets, that were inserted.
predics = model(output_np)
print('Number of Configurations/ One per batch element:', predics.shape)

(10, 20, 3)
torch.Size([10, 60])
<class 'torch.Tensor'>
Number of Configurations/ One per batch element: torch.Size([10])


## Permutation invariant architecture F 

In [5]:
class Phi(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.in_size = 3       #len(inputs[0,0,:])
        self.hidden1_size = 4
        self.hidden2_size = 8
        self.hidden3_size = 4
        self.out_size = 1
        
        self.lay1 = nn.Linear(self.in_size,self.hidden1_size)
        self.lay2 = nn.Linear(self.hidden1_size,self.hidden2_size)
        self.lay3 = nn.Linear(self.hidden2_size,self.hidden3_size)

        self.out = nn.Linear(self.hidden3_size,self.out_size)
     
    def forward(self, inputs):
        
        #batch_size, num_particles, space_dim = inputs.shape
        inputs = nn.functional.elu(self.lay1(inputs))
        inputs = nn.functional.elu(self.lay2(inputs))
        inputs = nn.functional.elu(self.lay3(inputs))
        out_phi = self.out(inputs)
        
        return out_phi

class Rho(nn.Module):
    def __init__(self, batch_size):
        super().__init__()
        
        self.in_size = batch_size #len(inputs[:]) #batch_size
        self.hidden1_size = 132
        self.hidden2_size = 164
        self.out_size = batch_size
        
        self.lay1 = nn.Linear(self.in_size,self.hidden1_size)
        self.lay2 = nn.Linear(self.hidden1_size,self.hidden2_size)
        
        self.out = nn.Linear(self.hidden2_size,self.out_size)
        
    def forward(self, inputs):
        
        #batch_size = len(inputs[:])
        inputs = nn.functional.elu(self.lay1(inputs))
        inputs = nn.functional.elu(self.lay2(inputs))
        out_rho = self.out(inputs)
        
        return out_rho

class F(nn.Module):
    def __init__(self, Phi, Rho, batch_size):
        super().__init__()
        self.model_phi = Phi
        self.model_rho = Rho
        
    def forward(self, inputs):
        # inputs is a torch tensor of shape (batch_size, 20, 3)
        #batch_size, num_particles, space_dim = inputs.shape
        transfer_data = self.model_phi(inputs) # processed through Phi
        #performing the summation over the 20 particles
        transfer_data = transfer_data.view(batch_size,-1).sum(1) 
        out = self.model_rho(transfer_data) # processed through Rho
        
        return out

batch_size = 100 # !!!-> NEEDS to be ADAPTED depending on INPUT <-!!!
phi = Phi()
print(phi)
rho = Rho(batch_size) 
print(rho)
f = F(phi,rho,batch_size)
print(f)

Phi(
  (lay1): Linear(in_features=3, out_features=4, bias=True)
  (lay2): Linear(in_features=4, out_features=8, bias=True)
  (lay3): Linear(in_features=8, out_features=4, bias=True)
  (out): Linear(in_features=4, out_features=1, bias=True)
)
Rho(
  (lay1): Linear(in_features=100, out_features=132, bias=True)
  (lay2): Linear(in_features=132, out_features=164, bias=True)
  (out): Linear(in_features=164, out_features=100, bias=True)
)
F(
  (model_phi): Phi(
    (lay1): Linear(in_features=3, out_features=4, bias=True)
    (lay2): Linear(in_features=4, out_features=8, bias=True)
    (lay3): Linear(in_features=8, out_features=4, bias=True)
    (out): Linear(in_features=4, out_features=1, bias=True)
  )
  (model_rho): Rho(
    (lay1): Linear(in_features=100, out_features=132, bias=True)
    (lay2): Linear(in_features=132, out_features=164, bias=True)
    (out): Linear(in_features=164, out_features=100, bias=True)
  )
)


## Some testing on random data

In [6]:
#defining the data for  test:
test_rand = np.random.normal(size=(100, 20, 3))
test_data = torch.from_numpy(test_rand.astype(np.float32))
print(type(test_data))
print(test_data.shape)
#applying the permutation invariant network
configurations = f(test_data)
print('Number of Configurations/ One per batch element:', configurations.shape)

<class 'torch.Tensor'>
torch.Size([100, 20, 3])
Number of Configurations/ One per batch element: torch.Size([100])
